# Computing pow and log in the smart contract

In [ ]:
import numpy as np

# Typing
from numbers import Integral
from sage.symbolic.expression import Expression
from sage.rings.rational import Rational
from sage.rings import real_mpfr
from typing import Iterable, Tuple, Union

def to_sage(fp: Fixedpoint) -> Rational:
    return Integer(fp._mantissa) * Integer(2) ^ Integer(fp._exponent)

## Reference values

We are using SageMath for symbolic calculations as much as possible, which allows us to maintain high precision.

In [ ]:
# Reference values are very symbolic
ref_base: Expression = exp(10^-4)  # instead of 1.0001: (1 + x ≈ exp(x) when x → 0)
print(f'Adjusted basepoint: {ref_base.n(digits=20)} (vs. 1.0001 in Uniswap)')
ref_sqrtbase: Expression = ref_base.sqrt()

## Simple log
We need to compute $\lfloor\log_{b}{\frac{x}{y}}\rfloor$ where $x$ and $y$ are relatively close ($\frac{x}{y} \in [0.7, 1.5]$).

In [ ]:
# The exact logarithm we are computing
x, y = var('x y')
ref_log = floor(log(x / y, ref_base))

Our base is deliberately chosen to be a power of $e$, which means that when calculating a logarithm, we can change base to natural and the denominator will simplify: $\log_{e^{1/c}}{a} = \frac{\ln{a}}{\ln{e^{1/c}}} = c \ln(a)$.

Now we only need to approximate $\lfloor\ln(\frac{x}{y})\rfloor$.
This approximation is due to Arthur, and we currently don’t quite understand how it works, we need to ask him.

Cool things that are supposed to be going on here:

* This approximation is really good (better than second-degree, worse than third?).
* It has one-sided error (required due to `floor`) (that’s probably where 0.0003 comes from).
* Somehow, all coefficients involved are integral. Is this some magic?

In [ ]:
def approx_log(x: Integral, y: Integral) -> Integral:
    x_plus_y: Integral = x + y
    num: Integral = 60003 * (x - y) * x_plus_y
    denom: Integral = 2 * (x_plus_y ^ 2 + 2 * x * y)
    return num // denom

In [ ]:
def test_log_approx(x: Integral, y: Integral) -> bool:
    return ref_log(x, y) == approx_log(x, y)

def gen_log_inputs() -> Iterable[Tuple[Integral, Integral]]:
    min_ratio = 0.7
    max_ratio = 1.5
    for y in range(1, 10^3):
        for ratio in np.linspace(min_ratio, max_ratio, 10, True):
            x = int(y * ratio)
            if not (min_ratio <= x/y <= max_ratio): continue
            yield (x, y)
            
for (x, y) in gen_log_inputs():
    assert test_log_approx(x, y), f'Failed for ({x}, {y}): {ref_log(x, y)} vs. {approx_log(x, y)}'

### Taylor series approximation

For any function $f$, degree $n$, and point $a$ we have:

$$f(x) = \sum\limits_{i=0}^n \frac{1}{n!} f^{(n)}(a) (x-a)^n + \frac{1}{(n + 1)!} f^{(n+1)}(c) (x - a)^{n+1}$$

The latter is the error term, where $c$ is some unknown point between $a$ and $x$. By getting some reasonable bound on the $(n+1)$-st derivative on this interval, we can get a bound on the approximation error.

In [ ]:
a, x, y, z = var('a x y z')
log(x / y)(x = (1 + a) * (x + y) / 2, y = (1 - a) * (x + y) / 2)
taylor(_, a, 1, 6)(a = x/y)

In [ ]:
taylor(log(x), x, 1, 5)

## Fast exponentiation

Our goal is to compute $b^i$ for arbitrary (but, hopefully, not too large) integer $i$.
There is a standard trick for fast exponentiation: [Exponentiation by squaring][wiki:fastexp].

The basic idea is that to compute $a^b$ we, first of all, represent the power in binary: $b = \sum\limits_{i=0}^{\lfloor\log_2{b}\rfloor} b_i 2^i$.
Then we use the fact that $a^{c+d} = a^c a^d$ to rewrite:

$$a^b = a^{\sum\limits_{i=0}^{\lfloor\log_2{b}\rfloor} b_i 2^i} = \prod\limits_{i=0}^{\lfloor\log_2{b}\rfloor} a^{b_i 2^i} = \prod\limits_{i=0}^{\lfloor\log_2{b}\rfloor} (a^{2^i})^{b_i}$$

Note that this is just the product of powers of $a$ corresponding to those bits of $b$ that are set and that those power are easy to compute by repeated squaring.
Here is this algorithm in Python:

[wiki:fastexp]: https://en.wikipedia.org/wiki/Exponentiation_by_squaring

In [ ]:
def fast_exp(a: Integral, b: Integral) -> Union[Integral, Rational]:
    result = 1
    power = a if b >= 0 else 1 / a
    if b < 0:
        b = -b
    while b > 0:
        if b % 2 == 1:
            result *= power
        b >>= 1  # warning: this is NOT a bind!
        power *= power
    return result

The algorithm above does $\lfloor\log_2{b}\rfloor + 1$ multiplications to compute the powers and multiplies a power into `result` for each bit of $b$ that is set.

If we are too lazy, we can precompute all $a^{2^i}$ for the range that we expect to be working with and save those $\mathcal{O}(\log b)$ multiplications. There are pros and cons of precomputing:

* pro: precomputing gets rid of a bunch of online multiplication;
* pro: we can precompute those values with just the right precision that we need, while multiplying online requires knowing $a$ with a high precision from the beginning;
* con: it will require storing and accessing those powers.

# WIP

`Fixedpoint` is a custom type for fixed-point arithmetic, except that the point is not really fixed as the scaling factor can change as a result of performing arithmetic operations. Also, values of this type track their error.

In [ ]:
from fixedpoint import Fixedpoint

# Target precision in bits
prec = 80

approx_sqrtbase = (ref_sqrtbase * 2^prec).round()
sqrtbase = Fixedpoint(int(approx_sqrtbase), -prec, Fixedpoint(1, -prec - 1))
print(f'Approximated sqrt(basepoint): {sqrtbase}')

# sanity check: we are supposed to be precise up to half of our scale
assert abs(to_sage(sqrtbase) - ref_sqrtbase).n(10*prec) < to_sage(sqrtbase._error)

---

In [ ]:
# Scratch space